# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

In [41]:
-- top_five_products_each_category
WITH ranked AS (
	SELECT 
		category,
		product_name,
		ROUND(SUM(sales)::NUMERIC, 2) AS product_total_sales,
		ROUND(SUM(profit)::NUMERIC, 2) AS product_total_profit,
		RANK() OVER (PARTITION BY category ORDER BY SUM(sales) DESC) AS product_rank
	FROM orders o
	JOIN products p
		USING (product_id)
	GROUP BY category, product_name
)
SELECT *
FROM ranked
WHERE product_rank <= 5
ORDER BY category ASC, product_total_profit DESC, product_rank


,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
1,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
2,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
3,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
4,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
5,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
6,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
7,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2


In [42]:
-- impute_missing_values
WITH empty_quantity AS (
	SELECT 
		product_id,
		discount,
		market,
		region,
		sales,
		quantity
	FROM orders
	WHERE quantity IS NULL
),
unit_price_calculation AS (
	SELECT 
		product_id,
		discount,
		AVG(sales/quantity) AS unit_price
	FROM orders
	WHERE quantity IS NOT NULL
	GROUP BY product_id, discount
)
SELECT DISTINCT
	e.*,
	e.sales/unit_price AS calculated_quantity 
FROM empty_quantity e
JOIN unit_price_calculation up
	ON e.product_id = up.product_id AND e.discount = up.discount

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4.0
1,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,2.0
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3.0
3,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2.0
4,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,4.0
